In [1]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import UpSampling2D, Conv2DTranspose, AveragePooling2D, ZeroPadding2D, Add, Conv2D, Flatten, Reshape
from pprint import pprint

from models.blocks import create_res_conv_block, create_input_layer, conv_block
from models.rangeview_branch import create_range_view_branch

In [2]:
bev_in = create_input_layer((800, 700, 10), 'bev')
x = bev_in

'''
    Backbone
'''
for i in range(2): # same res
    x = create_res_conv_block(x, 24, 3)
f1 = x
    
for i in range(4): # 1 / 2 res
    x = create_res_conv_block(x, 48, 3, i==0)
f2 = x

for i in range(4): # 1 / 4 res
    x = create_res_conv_block(x, 96, 3, i==0)
f3 = x

for i in range(4): # 1 / 8 res
    x = create_res_conv_block(x, 192, 3, i==0)
f4 = x

for i in range(4): # 1 / 16 res
    x = create_res_conv_block(x, 256, 3, i==0)
f5 = x

In [3]:
rv_net = create_range_view_branch(input_shape=(375, 1242, 3), 
                                  input_names=['depth_map_input', 'intensity_map_input', 'height_map_input'])
rv_inputs  = rv_net['inputs']
rv_outputs = rv_net['outputs']

l2, l3, l4, l5 = rv_outputs
l2, l3, l4, l5

(<tf.Tensor 'average_3/Identity:0' shape=(None, 188, 621, 24) dtype=float32>,
 <tf.Tensor 'average_5/Identity:0' shape=(None, 94, 311, 48) dtype=float32>,
 <tf.Tensor 'average_7/Identity:0' shape=(None, 94, 311, 48) dtype=float32>,
 <tf.Tensor 'average_9/Identity:0' shape=(None, 47, 156, 92) dtype=float32>)

In [7]:
'''
    RV FPN
'''
rv_l2_dw = conv_block(l2, 96, 1, 1)
rv_l2_dw = AveragePooling2D()(rv_l2_dw)
rv_l2_dw = ZeroPadding2D(((0,0), (2,0)))(rv_l2_dw)
rv_l2 = tf.keras.backend.resize_images(rv_l2_dw, 
                                       200/94,
                                       175/312,
                                       'channels_last',
                                       'bilinear')

rv_l3_zp = conv_block(l3, 96, 1, 1)
rv_l3 = tf.keras.backend.resize_images(rv_l3_zp, 
                                       200/94,
                                       175/311,
                                       'channels_last',
                                       'bilinear')

rv_l4_zp = conv_block(l4, 96, 1, 1)
rv_l4 = tf.keras.backend.resize_images(rv_l4_zp, 
                                       200/94,
                                       175/311,
                                       'channels_last',
                                       'bilinear')

rv_l5_up = conv_block(l5, 96, 1, 1)
rv_l5 = tf.keras.backend.resize_images(rv_l5_up, 
                                       200/47,
                                       175/156,
                                       'channels_last',
                                       'bilinear')

pprint([rv_l2, rv_l3, rv_l4, rv_l5])

# # rv_out = Add()([ rv_l2_dw, rv_l3_zp, rv_l4_zp, rv_l5_up ])
# # print(rv_out)
# # rv_out_f = tf.keras.backend.resize_images(rv_out, 
# #                                           200/94,
# #                                           175/312,
# #                                           'channels_last',
# #                                           'bilinear')

# # rv_out_f

[<tf.Tensor 'resize_6/ResizeBilinear:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'resize_7/ResizeBilinear:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'resize_8/ResizeBilinear:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'resize_9/ResizeBilinear:0' shape=(None, 200, 175, 96) dtype=float32>]


In [5]:
'''
    FPN + Header
'''
f2_dw = conv_block(f2, 96, 1, 1)
f2_dw = AveragePooling2D()(f2_dw)
f2_dw = Add()([f2_dw, rv_l2])

f3_zp = f3
f3_zp = Add()([f3_zp, rv_l3])

f4_up = conv_block(f4, 96, 1, 1)
f4_up = tf.keras.backend.resize_images(f4_up, 
                                       200/100,
                                       175/88,
                                       'channels_last',
                                       'bilinear')
f4_up = Add()([f4_up, rv_l4])

f5_up = conv_block(f5, 96, 1, 1)
f5_up = tf.keras.backend.resize_images(f5_up, 
                                       200/50,
                                       175/44,
                                       'channels_last',
                                       'bilinear')
f5_up = Add()([f5_up, rv_l5])

out = Add()([f2_dw, f3_zp, f4_up, f5_up])

obj_map = Conv2D(filters=1, 
                 kernel_size=1, 
                 padding='same', 
                 activation='sigmoid', 
                 name='obj_map', 
                 kernel_initializer='glorot_normal')(out)
geo_map = Conv2D(filters=11, 
                 kernel_size=1, 
                 padding='same', 
                 activation=None, 
                 name='geo_map',
                 kernel_initializer='glorot_normal')(out)

pprint([f2, f3, f4, f5])
print('-----------------')
pprint([f2_dw, f3_zp, f4_up, f5_up])
print('-----------------')
pprint([out])
print('-----------------')
pprint([obj_map, geo_map])
print('-----------------')

[<tf.Tensor 'add_5/Identity:0' shape=(None, 400, 350, 48) dtype=float32>,
 <tf.Tensor 'add_9/Identity:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'add_13/Identity:0' shape=(None, 100, 88, 192) dtype=float32>,
 <tf.Tensor 'add_17/Identity:0' shape=(None, 50, 44, 256) dtype=float32>]
-----------------
[<tf.Tensor 'add_36/Identity:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'add_37/Identity:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'add_38/Identity:0' shape=(None, 200, 175, 96) dtype=float32>,
 <tf.Tensor 'add_39/Identity:0' shape=(None, 200, 175, 96) dtype=float32>]
-----------------
[<tf.Tensor 'add_40/Identity:0' shape=(None, 200, 175, 96) dtype=float32>]
-----------------
[<tf.Tensor 'obj_map/Identity:0' shape=(None, 200, 175, 1) dtype=float32>,
 <tf.Tensor 'geo_map/Identity:0' shape=(None, 200, 175, 11) dtype=float32>]
-----------------


In [6]:
model = Model([bev_in] + rv_inputs, [obj_map, geo_map]).summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bev (InputLayer)                [(None, 800, 700, 10 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 800, 700, 10) 40          bev[0][0]                        
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 800, 700, 10) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 800, 700, 24) 2160        re_lu[0][0]                      
______________________________________________________________________________________________